In [1]:
import pandas as pd
import gtfs_functions as gtfs
from multiprocessing import  Pool
import numpy as np

pd.set_option('display.max_rows', 100)


In [335]:
apc_df =  pd.read_parquet('./carta-apc.parquet/2021_10_24_gtfs_days', engine='pyarrow').sort_values('transit_date').reset_index(drop=True)
routes, stops, stop_times, trips, shapes = gtfs.import_gtfs('./raw/20211024.zip', busiest_date=False)

In [356]:
apc_df = apc_df[apc_df['gtfs_trip_id']>'204888020']

In [434]:
# apc_df.route_id

In [355]:
trips.sort_values('trip_id')

,trip_id,route_id,service_id,direction_id,shape_id
0,204888020,1,1,0,shp-1-04
1,204889020,1,1,0,shp-1-03
2,204890020,1,1,0,shp-1-03
3,204891020,1,1,0,shp-1-03
4,204892020,1,1,0,shp-1-03
...,...,...,...,...,...
1093,207271010,33,2,1,shp-33-51
518,207272020,21,1,0,shp-21-02
542,207273020,21,1,1,shp-21-53
519,207274020,21,1,0,shp-21-01


In [138]:
# trips = trips[(trips.shape_id < 'shp-10C') | (trips.shape_id > 'shp-10D')]

In [38]:
# stop_df = pd.read_csv('./raw/20211024/stops.txt')

In [41]:
# len(stop_df.stop_id.unique())

1178

In [435]:
#works well to remove all null time_actual_arrive and associated stops
apc_df = apc_df.dropna(subset=['map_latitude', 'map_longitude'])
apc_df_w_10C = apc_df[(apc_df.route_id != '10C')]

In [16]:
# def parallelize_dataframe(df, func, n_cores=6):
#     df_split = np.array_split(df, n_cores)
#     pool = Pool(n_cores)
#     df = pd.concat(pool.map(func, df_split))
#     pool.close()
#     pool.join()
#     return df

In [15]:
# def remove_timepoints(df):
#     df[~df['stop_id'].astype(str).str.startswith('100')]
#     return df

In [17]:
# parallelize_dataframe(apc_df[apc_df.transit_date == '2022-01-11'],  remove_timepoints)

In [13]:
# apc_df[apc_df.transit_date == '2022-01-11'][~apc_df['stop_id'].astype(str).str.startswith('100')][['stop_id', 'gtfs_trip_id', 'time_actual_arrive','first_last_stop', 'timepoint', 'map_latitude']].sort_values('time_actual_arrive')[(apc_df.gtfs_trip_id == '205155020')]

In [125]:
# apc_df[(apc_df.gtfs_trip_id == '207272020') & (apc_df.transit_date == '2022-02-11')][['stop_id', 'time_actual_arrive','first_last_stop', 'timepoint', 'map_latitude', 'map_longitude']].sort_values('time_actual_arrive')

In [358]:
def check_stops_in_order(trip):
    trip = trip.sort_values('time_actual_arrive')
    if trip.iloc[0].first_last_stop !=1:
        return 1
    elif trip.iloc[-1].first_last_stop !=1:
        return 1
    else:
        return 0

In [129]:
# apc_df.sort_values(['trip_id', 'time_actual_arrive'])[['gtfs_trip_id', 'stop_id', 'gtfs_shape_id', 'time_actual_arrive']]

In [359]:
len(stops.stop_id.unique())

1122

In [183]:
# len(apc_df_w_10C.stop_id.drop_duplicates())
# apc_df_w_10C.gtfs_shape_id.unique()

In [360]:
def find_trips(series):
    return (series.to_list())

def stop_len(series):
    return (len(series.to_list()))

In [361]:
shape_alltrips = trips.groupby('shape_id').agg({"trip_id": find_trips}).reset_index()
shapes_10C = shape_alltrips[(shape_alltrips.shape_id >= 'shp-10C') & (shape_alltrips.shape_id <= 'shp-10D')]
#removing shape-10-C
shape_alltrips = shape_alltrips[~((shape_alltrips.shape_id >= 'shp-10C') & (shape_alltrips.shape_id <= 'shp-10D'))]

In [124]:
# shape_alltrips

In [362]:
apc_shape_alltrips = apc_df.groupby('gtfs_shape_id').agg({"gtfs_trip_id": find_trips}).reset_index()

In [363]:
trip_stop_len = apc_df_w_10C.groupby(['gtfs_shape_id', 'gtfs_trip_id', 'transit_date']).count().reset_index()

In [364]:
def func1(grp):
    grp = grp.sort_values('stop_id', ascending=False)
    return(grp.iloc[0])



In [365]:
trip_per_shape = trip_stop_len[['gtfs_shape_id', 'gtfs_trip_id', 'transit_date', 'stop_id']].groupby('gtfs_shape_id').apply(func1).reset_index(drop=True)

In [329]:
# trip_per_shape

In [190]:
# #determine the trip with max stop counts for each shape 
# for i, row in apc_shape_alltrips[:1].iterrows():
#     max = 0
#     for trip in row.gtfs_trip_id:
#         temp = apc_df[apc_df.gtfs_trip_id == trip].groupby('transit_date').count().reset_index()
#         temp2 = temp.iloc[temp.stop_id.idxmax()]
#         # print(temp.iloc[temp.stop_id.idxmax()])
#         if temp2.stop_id>max:
#             max = temp2.stop_id
#             apc_shape_alltrips.loc[i, 'max_stop_trip'] = trip
#             apc_shape_alltrips.loc[i, 'max_stop_date'] = temp2.transit_date
#             # break

In [366]:
#check if gtfs stops are matching with apc stops
gtfs_trip_stop_count = stop_times.groupby('trip_id').count().reset_index()[['trip_id', 'stop_id']]

In [367]:
merged_apc_gtfs = trip_per_shape.merge(gtfs_trip_stop_count, left_on='gtfs_trip_id', right_on='trip_id', how='left')

In [368]:
merged_apc_gtfs['diff'] = merged_apc_gtfs['stop_id_x'] - merged_apc_gtfs['stop_id_y']

In [369]:
diff = merged_apc_gtfs[merged_apc_gtfs['diff']!=0].rename({'stop_id_y': 'gtfs_stops', 'stop_id_x': 'apc_stops'}, axis=1)

In [372]:
diff

,gtfs_shape_id,gtfs_trip_id,transit_date,apc_stops,trip_id,gtfs_stops,diff


In [371]:
# shapes[shapes.shape_id=='']

In [373]:
for i, row in diff.iterrows():
    temp1 = (apc_df[(apc_df.gtfs_trip_id == (row.gtfs_trip_id)) & (apc_df.transit_date == row.transit_date)]).sort_values('time_actual_arrive')[['stop_id', 'time_actual_arrive', 'map_latitude']]
    temp2 = stop_times[stop_times.trip_id == row.trip_id].sort_values('stop_sequence')[['stop_id', 'stop_sequence', 'arrival_time']]
    res = temp1[~temp1.isin(temp2)]
    print(temp1, temp2)
    break

In [375]:
# merge10C
shapes = shapes[(shapes['shape_id'] <'shp-10C') | (shapes['shape_id'] >='shp-10D')]

In [378]:
routes2, stops2, stop_times2, trips2, shapes2 = gtfs.import_gtfs('./raw/2022-06-10.zip', busiest_date=False)


In [384]:
updated_10C = trips2[(trips2.shape_id == '1131953846') | (trips2.shape_id == '1131953847') | (trips2.shape_id == '1131953847')]

In [467]:
updated_10C

,trip_id,route_id,service_id,direction_id,shape_id
0,795f79b1-a-23226e03-8,203f5b65,1,1,1131953846
1,fbca78b9-e-23226e03-8,203f5b65,1,0,1131953847
2,0201b206-8-23226e03-8,203f5b65,1,0,1131953847


In [472]:
stop_times_10C = stop_times2.merge(updated_10C, on='trip_id', how='right').sort_values(['trip_id', 'arrival_time'])

In [473]:
stop_times_10C

,trip_id,stop_id,stop_sequence,arrival_time,departure_time,timepoint,shape_dist_traveled,route_id_x,service_id_x,direction_id_x,shape_id_x,stop_code,stop_name,geometry,route_id_y,service_id_y,direction_id_y,shape_id_y
103,0201b206-8-23226e03-8,2234,1,68760.0,68760.0,1,3003.217828,203f5b65,1,0,1131953847,3230,Highway 58 & Oakwood Dr -1-0,POINT (-85.18691 35.08955),203f5b65,1,0,1131953847
104,0201b206-8-23226e03-8,225,2,68820.0,68820.0,0,3566.487227,203f5b65,1,0,1131953847,1895,HIGHWAY 58 + WEBB ROAD,POINT (-85.19172 35.08641),203f5b65,1,0,1131953847
105,0201b206-8-23226e03-8,2233,3,68820.0,68820.0,0,3742.706828,203f5b65,1,0,1131953847,2945,Highway 58 & Trailwood,POINT (-85.19343 35.08567),203f5b65,1,0,1131953847
106,0201b206-8-23226e03-8,302,4,68940.0,68940.0,0,7730.556980,203f5b65,1,0,1131953847,1279,BONNY OAKS + TUGALOO,POINT (-85.23328 35.07451),203f5b65,1,0,1131953847
107,0201b206-8-23226e03-8,301,5,68940.0,68940.0,0,7871.026546,203f5b65,1,0,1131953847,1278,BONNY OAKS + PARKWAY,POINT (-85.23471 35.07403),203f5b65,1,0,1131953847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,fbca78b9-e-23226e03-8,989,44,25740.0,25740.0,0,15744.459787,203f5b65,1,0,1131953847,2243,MCCALLIE AVE + GREENWOOD,POINT (-85.28129 35.03836),203f5b65,1,0,1131953847
99,fbca78b9-e-23226e03-8,990,45,25800.0,25800.0,0,15876.032557,203f5b65,1,0,1131953847,2248,MCCALLIE AVE + HOLTZCLAW,POINT (-85.28261 35.03885),203f5b65,1,0,1131953847
100,fbca78b9-e-23226e03-8,991,46,25800.0,25800.0,0,16078.598334,203f5b65,1,0,1131953847,2261,MCCALLIE AVE + WARNER PK,POINT (-85.28462 35.03963),203f5b65,1,0,1131953847
101,fbca78b9-e-23226e03-8,992,47,25800.0,25800.0,0,16566.326977,203f5b65,1,0,1131953847,2256,MCCALLIE AVE + ONEAL,POINT (-85.28952 35.04139),203f5b65,1,0,1131953847


In [441]:
trips_txt = pd.read_csv('./raw/20211024/trips.txt')
stop_times_txt = pd.read_csv('./raw/20211024/stop_times.txt')
stop_times_txt.trip_id = stop_times_txt.trip_id.astype(str)

In [442]:
trips_txt_10C = pd.read_csv('./raw/20211024/trips.txt')
trips_txt_10C = trips_txt_10C[(trips_txt_10C['route_id'] == '10C')]
trips_txt_10C.trip_id = trips_txt_10C.trip_id.astype(str)

In [468]:
stop_times_txt_10C = stop_times_txt.merge(trips_txt_10C, on='trip_id', how='inner')
# trips_txt_10C

In [454]:
def time_convert(x):
    times = x.split(':')
    return ((3600*int(times[0])+60*int(times[1]))+int(times[2]))

In [469]:
stop_times_txt_10C['arrival_time'] = stop_times_txt_10C['arrival_time'].apply(time_convert)

In [470]:
stop_times_txt_10C.sort_values(['trip_id', 'arrival_time'])[stop_times_txt_10C.stop_sequence == 1]

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint,route_id,service_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed,block_name
0,205136020,66300,18:25:00,1940,1,NaN,0,0,0.0,1,10C,1,CAMPBELL-UTC HIGHWAY 58,NaN,0,2402,shp-10C-52,2,2,1519
70,205137020,59760,16:36:00,2092,1,NaN,0,0,0.0,1,10C,1,DOWNTOWN CAMPBELL-UTC HIGHWAY 58,NaN,0,2402,shp-10C-51,2,2,1519
143,205138020,57060,15:51:00,2092,1,NaN,0,0,0.0,1,10C,1,DOWNTOWN CAMPBELL-UTC HIGHWAY 58,NaN,0,3102,shp-10C-51,2,2,159
216,205139020,57000,15:50:00,1940,1,NaN,0,0,0.0,1,10C,1,CAMPBELL-UTC HIGHWAY 58,NaN,0,7402,shp-10C-52,2,2,905
286,205140020,27000,07:30:00,1940,1,NaN,0,0,0.0,1,10C,1,CAMPBELL-UTC HIGHWAY 58,NaN,0,402,shp-10C-52,2,2,1018
356,205141020,28800,08:00:00,1940,1,NaN,0,0,0.0,1,10C,1,CAMPBELL-UTC HIGHWAY 58,NaN,0,3702,shp-10C-52,2,2,302
426,205142020,47760,13:16:00,2092,1,NaN,0,0,0.0,1,10C,1,DOWNTOWN CAMPBELL-UTC HIGHWAY 58,NaN,0,2302,shp-10C-51,2,2,1515
499,205143020,22860,06:21:00,2092,1,NaN,0,0,0.0,1,10C,1,DOWNTOWN CAMPBELL-UTC HIGHWAY 58,NaN,0,202,shp-10C-51,2,2,1015
572,205144020,20160,05:36:00,2092,1,NaN,0,0,0.0,1,10C,1,DOWNTOWN CAMPBELL-UTC HIGHWAY 58,NaN,0,402,shp-10C-51,2,2,1018
645,205145020,17160,04:46:00,2092,1,NaN,0,0,0.0,1,10C,1,DOWNTOWN CAMPBELL-UTC,NaN,0,5202,shp-10C-53,2,2,405


In [480]:
stop_times_txt_10C[['trip_id', 'arrival_time', 'shape_dist_traveled']].merge(stop_times_10C, on=['arrival_time'], how='inner')

,trip_id_x,arrival_time,shape_dist_traveled_x,trip_id_y,stop_id,stop_sequence,departure_time,timepoint,shape_dist_traveled_y,route_id_x,service_id_x,direction_id_x,shape_id_x,stop_code,stop_name,geometry,route_id_y,service_id_y,direction_id_y,shape_id_y
0,205144020,20160,0.00,795f79b1-a-23226e03-8,308,51,20160.0,0,18311.307947,203f5b65,1,1,1131953846,1885,HIGHWAY 58 + BRIARWOOD,POINT (-85.19296 35.08569),203f5b65,1,1,1131953846
1,205144020,20160,0.00,795f79b1-a-23226e03-8,309,52,20160.0,0,18434.144472,203f5b65,1,1,1131953846,1888,HIGHWAY 58 + HAL DRIVE,POINT (-85.19193 35.08613),203f5b65,1,1,1131953846
2,205144020,20160,0.00,795f79b1-a-23226e03-8,310,53,20160.0,0,19543.470469,203f5b65,1,1,1131953846,1886,HIGHWAY 58 + FOOD LION,POINT (-85.18263 35.09199),203f5b65,1,1,1131953846
3,205147020,25500,52589.16,fbca78b9-e-23226e03-8,299,6,25500.0,0,8522.060714,203f5b65,1,0,1131953847,1419,CAMPBELL + BATTERY HEIGHTS,POINT (-85.24139 35.07243),203f5b65,1,0,1131953847
4,205147020,25500,52589.16,fbca78b9-e-23226e03-8,2222,7,25500.0,1,9169.658781,203f5b65,1,0,1131953847,12,Stuart & Chamberlain,POINT (-85.24634 35.07037),203f5b65,1,0,1131953847
5,205147020,25500,52589.16,fbca78b9-e-23226e03-8,292,8,25500.0,0,9299.747381,203f5b65,1,0,1131953847,1481,CHAMBERLAIN + CUSHMAN,POINT (-85.24692 35.06930),203f5b65,1,0,1131953847
6,205147020,25500,52589.16,fbca78b9-e-23226e03-8,351,9,25500.0,0,9403.444446,203f5b65,1,0,1131953847,1470,CHAMBERLAIN + BOONE,POINT (-85.24735 35.06843),203f5b65,1,0,1131953847
7,205147020,25500,52589.16,fbca78b9-e-23226e03-8,294,10,25500.0,0,9557.256690,203f5b65,1,0,1131953847,1485,CHAMBERLAIN + FAIRLEIGH,POINT (-85.24811 35.06721),203f5b65,1,0,1131953847
